In [1]:
import sys
sys.path = ['..'] + sys.path

In [2]:
import datetime
import glob
import shutil
import iris
import os
import numpy as np
import tiledb

import json

from tiledb_xarray.iris_writer import cubes_into_datasets,TileDBBuilder,TileDBAppender,linear_arr_extend

In [3]:
import importlib
import tiledb_xarray.iris_writer
importlib.reload(tiledb_xarray.iris_writer)
from tiledb_xarray.iris_writer import cubes_into_datasets,TileDBBuilder,TileDBAppender,linear_arr_extend

In [4]:
def log(*args):
    print(datetime.datetime.now(), *args)

In [5]:
NC_SOURCE_DIR = "/data/cssp-china/mini-dataset-24-01-19/nc/6hrly/"
files = [os.path.join(NC_SOURCE_DIR, f) for f in sorted(os.listdir(NC_SOURCE_DIR))][:1000]
# part1_files, part2_files = files[:1], files[1:]
# file_groups = [files[i:i+2] for i in range(0, len(files), 2)]

TILEDB_PATH = os.path.abspath('/data/cssp-china/theo-6hrly-tile-db')
TILEDB_PATH

'/data/cssp-china/theo-6hrly-tile-db'

In [27]:
# shutil.rmtree(TILEDB_PATH, ignore_errors=True)

In [6]:
log('load in iris')
first_file = files.pop(0)

2020-02-26 14:08:51.336239 load in iris


In [33]:
cubes = iris.load(first_file)
log('sort into domains')
datasets = cubes_into_datasets(cubes)
datasets

2020-02-26 11:41:41.198503 load in iris
2020-02-26 11:41:44.362907 sort into domains


In [34]:
%%time
log('Build tiledb')
builder = TileDBBuilder(datasets, TILEDB_PATH)
builder.build()
log('Built')

2020-02-26 11:41:46.724929 Build tiledb
2020-02-26 11:41:53.236727 insert into m01s03i237_m01s03i245_m01s05i217_m01s05i217 shape (16, 219, 286)
2020-02-26 11:42:07.461953 insert into m01s16i202_m01s16i203 shape (17, 14, 219, 286)
2020-02-26 11:44:09.053879 insert into m01s12i202_m01s15i201_m01s15i202 shape (17, 14, 218, 286)
2020-02-26 11:47:04.329094 insert into m01s00i024 shape (17, 219, 286)
2020-02-26 11:47:15.661655 insert into m01s03i225_m01s03i226 shape (17, 218, 286)
2020-02-26 11:47:25.587779 Built


In [38]:
%%time
#  Extend in time....
log('expand times')
time_arrs = glob.glob(TILEDB_PATH+'/**/time')
for arr in time_arrs:
    linear_arr_extend(arr, 1000)
    log('Done', os.path.basename(arr))

2020-02-26 11:52:20.311642 expand times
2020-02-26 11:52:25.099528 Done time
2020-02-26 11:52:27.238459 Done time
2020-02-26 11:52:29.385417 Done time
2020-02-26 11:52:31.412611 Done time
2020-02-26 11:52:33.711201 Done time
CPU times: user 84.3 ms, sys: 90.7 ms, total: 175 ms
Wall time: 13.4 s


In [11]:
import dask
import dask.bag as db
    
        

In [12]:
def convert(file):
    with dask.config.set(scheduler='single-threaded'):
        cubes = iris.load(file)
        datasets = cubes_into_datasets(cubes)
        for dataset in datasets:
            log(f'   add ds {dataset.key}')
            appender = TileDBAppender(dataset, TILEDB_PATH)
            appender.build()
    return (file)

In [13]:
conversion  = db.from_sequence(files[:1]).map(convert)

In [ ]:
conversion.compute()

2020-02-26 14:09:36.340973    add ds m01s03i237_m01s03i245_m01s05i217_m01s05i217
2020-02-26 14:09:37.421434 insert into m01s03i237_m01s03i245_m01s05i217_m01s05i217 shape (40, 219, 286)
2020-02-26 14:10:00.296306    add ds m01s16i202_m01s16i203
2020-02-26 14:10:02.071419 insert into m01s16i202_m01s16i203 shape (40, 14, 219, 286)


In [14]:
from dask.distributed import Client, progress
import distributed
import dask
from dask_kubernetes import KubeCluster
from dask import bag as db

from dask.diagnostics import ProgressBar

cluster = KubeCluster()
cluster.scale_up(50)
display(cluster)
# Connect dask to the cluster
client = distributed.Client(cluster)
client

distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://10.244.0.252:39299
distributed.scheduler - INFO -   dashboard at:                     :8787


KubeCluster('tcp://10.244.0.252:39299', workers=0, threads=0, memory=0 B)

distributed.scheduler - INFO - Receive client connection: Client-607db8f0-589d-11ea-8035-26577c477f88
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.244.0.252:39299 Dashboard: /user/tam203/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.0.253:40679', name: 17, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.253:40679
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.0.254:44241', name: 42, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.254:44241
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.0.3:36907', name: 25, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.3:36907
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.9.195:35889', name: 20, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.9.195:35889
di

In [25]:
# # Append the rest
# # for file in part2_files:
# log('load rest of the cubes ')
# for i, file in enumerate(files):
#     log(f"add file {i}")
#     cubes = iris.load(file)
#     datasets = cubes_into_datasets(cubes)
#     for dataset in datasets:
#         log(f'   add ds {dataset.key}')

#         appender = TileDBAppender(dataset, TILEDB_PATH)
#         appender.build()


# log("Fin")

2020-02-26 11:28:30.352205 load rest of the cubes 
2020-02-26 11:28:30.352610 add file 0
2020-02-26 11:28:35.314672    add ds m01s03i237_m01s03i245_m01s05i217_m01s05i217
2020-02-26 11:28:35.512889 insert into m01s03i237_m01s03i245_m01s05i217_m01s05i217 shape (40, 219, 286)
2020-02-26 11:28:46.124416    add ds m01s16i202_m01s16i203
2020-02-26 11:28:47.083419 insert into m01s16i202_m01s16i203 shape (40, 14, 219, 286)


KeyboardInterrupt: 

In [19]:

conversion = conversion.persist()  # start computation in the background
progress(conversion)      # watch progress

VBox()

In [16]:
%conda install ipywidgets -y

done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/my-conda-envs/py3.7

  added / updated specs:
    - ipywidgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bleach-3.1.0               |           py37_0         220 KB
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2019.11.28         |           py37_0         153 KB
    defusedxml-0.6.0           |             py_0          23 KB
    entrypoints-0.3            |           py37_0          12 KB
    gmp-6.1.2                  |       h6c8ec71_1         514 KB
    importlib_metadata-1.5.0   |           py37_0          48 KB
    ipywidgets-7.5.1           |             py_0         107 KB
    jsonschema-3.2.0           | 

distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.244.0.10:34695', name: 3, memory: 7, processing: 11>
distributed.core - INFO - Removing comms to tcp://10.244.0.10:34695
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.0.10:33931', name: 3, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.10:33931
distributed.core - INFO - Starting established connection


notebook-6.0.3       | 4.0 MB    | ##################################### | 100% 
zipp-2.2.0           | 12 KB     | ##################################### | 100% 
webencodings-0.5.1   | 19 KB     | ##################################### | 100% 
pyrsistent-0.15.7    | 93 KB     | ##################################### | 100% 
widgetsnbextension-3 | 862 KB    | ##################################### | 100% 
jsonschema-3.2.0     | 95 KB     | ##################################### | 100% 
gmp-6.1.2            | 514 KB    | ##################################### | 100% 
nbformat-5.0.4       | 89 KB     | ##################################### | 100% 
certifi-2019.11.28   | 153 KB    | ##################################### | 100% 
mistune-0.8.4        | 55 KB     | ##################################### | 100% 
pandoc-2.2.3.2       | 14.0 MB   | ###############1                      |  41% 

distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.2.6:45901', name: 9, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.2.6:45901
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.2.8:42285', name: 14, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.2.8:42285
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.2.9:41315', name: 43, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.2.9:41315


pandoc-2.2.3.2       | 14.0 MB   | ################2                     |  44% 

...

pandoc-2.2.3.2       | 14.0 MB   | ####################################1 |  98% 

distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.1.13:33451', name: 34, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.1.13:33451
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.1.6:40013', name: 31, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.1.6:40013
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.1.7:46417', name: 44, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.1.7:46417
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.1.14:43765', name: 5, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.1.14:43765
distributed

pandoc-2.2.3.2       | 14.0 MB   | ##################################### | 100% 
terminado-0.8.3      | 26 KB     | ##################################### | 100% 
defusedxml-0.6.0     | 23 KB     | ##################################### | 100% 
bleach-3.1.0         | 220 KB    |                                       |   0% 

distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.4.8:44571', name: 27, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.4.8:44571
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.4.7:44305', name: 6, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.4.7:44305
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.4.6:36875', name: 47, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.4.6:36875
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.244.3.8:34043', name: 10, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.3.8:34043
distributed.cor

bleach-3.1.0         | 220 KB    | ##################################### | 100% 
prometheus_client-0. | 42 KB     | ##################################### | 100% 
ipywidgets-7.5.1     | 107 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [20]:
cluster.close()

distributed.scheduler - INFO - Scheduler closing...
distributed.scheduler - INFO - Scheduler closing all comms
distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.244.0.253:40679', name: 17, memory: 1, processing: 1>
distributed.core - INFO - Removing comms to tcp://10.244.0.253:40679
distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.244.0.254:44241', name: 42, memory: 0, processing: 0>
distributed.core - INFO - Removing comms to tcp://10.244.0.254:44241
distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.244.0.3:36907', name: 25, memory: 0, processing: 0>
distributed.core - INFO - Removing comms to tcp://10.244.0.3:36907
distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.244.0.4:40225', name: 16, memory: 0, processing: 0>
distributed.core - INFO - Removing comms to tcp://10.244.0.4:40225
distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.244.0.6:43979', name: 29, memory: 0, processing: 0>
distributed.core - INFO - Removing com